In [1]:
import pandas as pd
import numpy as np
import os 
import sys
import glob

In [2]:
dpath = "/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/"
file_list = glob.glob(f"{dpath}/*")
# file_list = file_list[:10]
file_list[:10]

['/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o3b58.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o1b33.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o4b01.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o4b23.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o2b48.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o4b08.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o1b35.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o3b30.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shared_data/single_cell/reports/read_lengths/o1b80.read_lengths.parquet',
 '/scratch/indikar_root/indikar1/shar

In [9]:
result = []

for file_path in file_list:
    basename = os.path.basename(file_path)
    sample_id = basename[:2]
    cell_id = basename[2:5]
    
    tmp = pd.read_parquet(file_path)
    
    row = {
        'sample_id' : sample_id,
        'cell_id' : cell_id,
        'total_reads' : len(tmp),
        'mean_read_length' : tmp['read_length'].mean(),
        'median_read_length' : tmp['read_length'].median(),
    }
    result.append(row)
    
result = pd.DataFrame(result)
result.head()

,sample_id,cell_id,total_reads,mean_read_length,median_read_length
0,o3,b58,127189,1538.684603,1265.0
1,o1,b33,461331,1081.535089,1093.0
2,o4,b01,16284,36.390629,21.0
3,o4,b23,1764,94.596939,24.0
4,o2,b48,3221,185.431543,10.0


In [11]:
gx = result.groupby('sample_id')['median_read_length'].mean()
gx

sample_id
o1    1036.765625
o2     579.869565
o3    1140.895833
o4     578.187500
Name: median_read_length, dtype: float64

In [4]:
tmp.head()

,read_name,read_length
0,6d628642-2ce0-4a23-8cc6-0d7d72561c77,840
1,721d9c3f-61f4-4134-b329-6a7d9a48a95b,1057
2,e58c69dc-4a05-4418-94f1-099cb5c1dbfd,1888
3,316362dd-8f07-42d3-8bf5-7ea0f30590d2,1351
4,5d635f28-caf6-4d9e-9b9f-648e9e36bfba,718


In [3]:
def get_read_lengths(df: pd.DataFrame) -> pd.DataFrame:
    """
    Efficiently calculates total and mapped read lengths for each read name.

    Args:
        df: A DataFrame with columns 'read_name', 'length_on_read', and 'is_mapped'.

    Returns:
        A DataFrame with columns 'read_name', 'bases', and 'bases_mapped'.
    """
    # Use groupby and aggregation
    total_lengths = df.groupby("read_name")["length_on_read"].sum()
    mapped_lengths = df[df["is_mapped"]].groupby("read_name")["length_on_read"].sum()

    # Combine results using efficient join
    result_df = pd.concat([total_lengths, mapped_lengths], axis=1).reset_index()
    result_df.columns = ["read_name", "bases", "bases_mapped"]

    # Impute missing values with 0 (if read has no mapped lengths)
    result_df["bases_mapped"] = result_df["bases_mapped"].fillna(0)
    result_df["bases_mapped"] = result_df["bases_mapped"].astype(int)

    return result_df

result = []
columns = [
    'read_name',
    'length_on_read',
    'is_mapped',
]

for file_path in file_list:
    basename = os.path.basename(file_path)
    sample_id = basename[:2]
    cell_id = basename[2:5]
    
    tmp = pd.read_parquet(file_path, columns=columns)
    tmp = get_read_lengths(tmp)
    n_reads_unmapped = tmp[tmp['bases_mapped'] == 0]
    
    row = {
        'sample_id' : sample_id,
        'cell_id' : cell_id,
        'total_reads' : len(tmp),
        'reads_unmapped' : len(n_reads_unmapped),
        'total_bases' : tmp['bases'].sum(),
        'total_bases_mapped' : tmp['bases_mapped'].sum(),
        'mean_read_length' : tmp['bases'].mean().astype(int),
        'median_read_length' : tmp['bases'].median().astype(int),
    }
    
    result.append(row)
    
    
result = pd.DataFrame(result)
result.head()

,sample_id,cell_id,total_reads,reads_unmapped,total_bases,total_bases_mapped,mean_read_length,median_read_length
0,o1,b18,604302,5993,637848243,571910866,1055,1023
1,o2,b03,1855,1796,115865,73897,62,16
2,o4,b04,22499,6604,18929423,18008085,841,674
3,o4,b79,7596,7395,435755,250163,57,7
4,o3,b86,21609,1234,24276003,21720873,1123,1124


In [4]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [7]:
gx = result.groupby('sample_id').agg(
    median_read_length = ('median_read_length', 'mean'),
).reset_index()

gx

,sample_id,median_read_length
0,o1,1036.750000
1,o2,579.858696
2,o3,1140.895833
3,o4,578.177083


In [ ]:
tmp.head()

In [ ]:
fpath = f"/scratch/indikar_root/indikar1/shared_data/single_cell/align_table/o4b12.GRCm39.align_table.parquet"

df = pd.read_parquet(fpath)
df.head()



In [ ]:
df[df['read_name'] == 'b32eaeb1-60df-4c53-bc22-d0b99737b0d4_1']


In [ ]:
df[df['read_name'] == 'b32eaeb1-60df-4c53-bc22-d0b99737b0d4_1']['length_on_read'].sum()

In [ ]:
len("CAGTTGGGGCATCTTCTGGATAAATGCCGAGGAGAGGTATTGCTGGATCCTCCGGTAGTACTATGTCCAAGTCTGAGGAACCTCAGACTGATTTCCAGAGTGGTTGTACAAGCCCGCAATCCCACCAACAATGGAGGAGTGTTCCTCTTTCTCCACATCCACGCCAGCATCTGCTGTCACCTGAATTTTTGATCTTAGCCATTCTGACTGGTGTGAGGTGGAATCTCAGGGTTGTTTTCATTTTGCATTTCCCTGATAATTAAGGATGTTGAACATTTTTCAGGTGCTTCTGTTCCATTCGGTATTTATAGGTGAGAATTCTGTGTTCATTCTGAGCCCCCCATTTTTAATAGGATTTTGAGATTTTCTGAAGTCCACCTTCTTCAGTTTCTTTATATATGTTGATATTAGCCCCCTATCTGACTTAGGATAGGTAAAGATCCTTTCCCAATCTGTTGGTGGTCTTTTTGTCTTATTGACGGTGTCTTTTGCCTTGCAGAAACTTTGGAGTTCATTAGGTCCCATTTGTCAAATTATCAATCTTACAGCACATGTGGAAACAGAAAAGAAAATCTGAAACAAAATCATAAGAAGAGGTGAAGTGAGAGTAGAAAAATGGGGAGATGGTAAATGCAGAATACGTGAATGTATAAAAATAGTAAAGTGTGTCCCCTTAATTCTAATAACCACTGATAATTATAGATAAATATTTCTAAGTAAAGAGGAAATGATTTAAACCTTTCACTGTCCTTGCAGAAGTATTTTAACACTTAGTATAATTTGATGAGGGAATCCAATTTTAAAACAGCCAAACTTATTTTAAATCTACCAAAGAGACTGCATTTTTAATACTGTCTTGTTAATGCAAAATATTTATTTTCCAGTTCCAACCTATTGATTCTAAAAAGATTAAGATATTATAGTGGCCAAGCTCTAAAAGTCTCTATAACATTCCCAAATACTTCATAGACATTGACTGAAATCTATTCATTGCTTAGAAATGACAAAGTTAAACCTATGTCCACAGTCAAAAGGTAGAGGGATAAATGCTGTTGCTTTGCTGGGTTTTCCTTATTCATCCAATTCAATACAGATGCCACCTGTGGGGTGGTATGTGTTCCCTCCTTTTGCCTCTCTGCAAACATGGGAACATCTCTATCCCTCCTTTTGCCTCTCTGCAAACTCCTTGAGAGACTGCACACCAACACACACACATTCAATCATCTCTTTTATGGGGACTAATATTCCAGATCTCCAATACCAGTGGAGTCTATTTTGGGACAGTTTATGCCTTATTTCATGTGTTTTCTCTGTATTTTTGTTCTTACCAGTTCAACATTTCATATTTTATGTTGTTTGTCTTTTTAATATCTATTTTGAGCTTCTTAATTTTTTTTAGAAACTGGATTTATTTTCATTTTTATATATGGATACCCTAGTATAACAGCAAGTCTCCGCAATTTTATATTTACCACTTCTCCTCTTACATCTTAACTCTATTACTCTTCCTGTGGGTTACATAAGAGCAAAGGTCTCTATTTTGTCTGTGGCTTTAGAACAGTGCAAAATGTAATAGTGGGCATTTAAAAATAGATTTGTACTCCAGTCAGTTCCAGAAGAAATTCTGTGTTTTCTTTATCTCAAGGCCTATTTCAGATGGCTTATGATTTTAAAATTTATATTCCTTCCTGTTTGGTATTTTTTTTCTTGTAACTCTCTAAGGCATTTTTTTTTGTTTCCTCTTTAAATGTGTCTAGCTGTTTACCTGTGTTTTCCTATATTTCCTTCTTAAAGTCCTCCATCATCACCATAAGAAGTGACTTTAGATCCGTGTCTTGCTTTTCTGGTGTGATCAAGTCGTTCAGTAAAAGTAAGGTTAAGTTAGCCGGGAGCTGGGGAAATGGCTCAGCAATTGAGAGTGCTTGTTCTTCTTACAGAGGACCTGGACTGGGTCCCTAGCACCTGTGCTGCATCTTTGTAAGTCCAGTTTTGGGGATGCAAGCCTTGTTTTGACCCTCAAGTATACCAGGAGTGTATGTAATCCTGAA")